# Modelo de Black-Litterman

El modelo propuesto va a permitir incluir información cualitativa en el CAPM. Se asume que la matriz de covarianza es dada y conocida, y nos centramos en las primas de riesgo. Lógicamente desde la perspectiva del CAPM el modelo es inutil pues apuesta claramente por una estrategia pasiva. Una estrategia de inversión activa precisa de previsiones sobre la evolución de alguno de los títulos. 

### El modelo de Black-Litterman: el papel del CAPM y de la estadística bayesiana.

Recordamos que las ponderaciones de una cartera eficiente vienen definidas por esta ecuación:

$$w^*=\frac{1}{\lambda}\Sigma^{-1}\pi \tag{1}$$

Definimos $w_{Mj}$ como la ponderación del activo $j$ en la cartera de mercado $M$. Por tanto la rentabilidad de la cartera de mercado viene definida por:

$$\tilde{r}_M=\sum_{j=1}^n w_{Mj}\tilde{r}_j \tag{2}$$

De acuerdo al CAPM:

$$\mu_j-r_f=\beta_j(\mu_M-r_f) \tag{3}$$

donde

$$\mu_M=\sum_{j=1}^n w_{Mj}\mu_j \tag{4}$$

El premio por el riesgo será:

$$\pi_{ej}=\mu_j-r_f \tag{4}$$

el subíndice $e$ se utiliza para enfatizar que se trata de un mercado en equilibrio. Si desarrollamos la ecuación anterior:

$$\begin{gather}
\pi_{ej}=\frac{Cov(\tilde{r}_i,\tilde{r}_M)}{\sigma_M^2}(\mu_M-r_f) \tag{5} \\
=\frac{1}{\sigma_M^2}Cov\left(\tilde{r}_i,\sum_{j=1}^{n} w_{Mj}\tilde{r}_j \right)(\mu_M-r_f) \\ 
=\frac{\mu_M-r_f}{\sigma_M^2} \sum_{j=1}^{n} w_{Mj} Cov(\tilde{r}_i,\tilde{r}_j ) 
\end{gather}$$

Podemos expresar matricialmente la ecuación 5:

$$\pi_e=\delta\Sigma w_M \tag{6}$$

y definimos el precio de mercado del riesgo

$$\delta=\frac{\mu_M-r_f}{\sigma_M^2} \tag{7}$$

Si aplicamos la estadística Bayesiana la prima por riesgo $pi_e$ es considerada como un vector de variables aleatorias $\tilde{\theta}$ condicionada por la información disponible $\mathcal{M}_e$, por tanto:

$$\pi_e=E\left[\tilde{\theta}\left. \right|\mathcal{M}_e\right] \tag{8}$$

La notación $\pi_S$ representa la prima por riesgo subjetiva y $\pi{BL}$ es la prima por riesgo propuesta por Black-Literrman. Supnomes que la rentabilidad y los parámetros se distribuyen como normales.

Vamos a suponer que el inversor cree que el activo 2 batirá la rentabilidad del activo 5 en un 2%, es decir:

